In [1]:
import os
import dicom
import cv2
import ijroi
import numpy as np
from matplotlib import pyplot, cm

/home/user01/anaconda2/envs/tensorflow/lib/python2.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
numOfDataSet = 10
numOfData = (19, 19, 19, 19, 21, 19, 21, 21, 19, 19)
numOfRoiData = (3, 7, 3, 8, 2, 6, 10, 2, 6, 9)
refRoiSetNum = ('0000', '0004', '0014', '0015', '0016', '0019', '0026', '0028', '0035', '0037')

PathDicom = "./data/"
lstFilesDCM = []  # create an empty list
lstFilesROI = []
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))
        if ".roi" in filename.lower():
            lstFilesROI.append(os.path.join(dirName,filename))

In [23]:
###### Get DCM file
# Get ref file
RefDs = dicom.read_file(lstFilesDCM[0])

# Load dimensions based on the number of rows, columns, ans slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM))

# load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))


x = np.arange(0.0, (ConstPixelDims[0]+1)*ConstPixelSpacing[0], ConstPixelSpacing[0])
y = np.arange(0.0, (ConstPixelDims[1]+1)*ConstPixelSpacing[1], ConstPixelSpacing[1])
z = np.arange(0.0, (ConstPixelDims[2]+1)*ConstPixelSpacing[2], ConstPixelSpacing[2])


# The array is sized based on 'ConstPixelDims'
tmpArrayDicom = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

# loop through all the DICOM files
for filenameDCM in lstFilesDCM:
    # read the file
    ds = dicom.read_file(filenameDCM)
        
    # store the raw image data
    tmpArrayDicom[:, :, lstFilesDCM.index(filenameDCM)] = ds.pixel_array

# reshape ArrayDicom (:, :, totalDataNum) -> (:, :, setNum, dataNum)
ArrayDicom = np.zeros([RefDs.Rows, RefDs.Columns, numOfDataSet, max(numOfData)], dtype=RefDs.pixel_array.dtype)
cnt = 0

for s in range(numOfDataSet):
    for i in range(numOfData[s]):
        ArrayDicom[:, :, s, i] = tmpArrayDicom[:, :, cnt+i]
    cnt = cnt + numOfData[s]
    
# Print Image
#for i in range(numOfData[0]):
#    print i
#    pyplot.figure(dpi=300)    
#    pyplot.axes().set_aspect('equal', 'datalim')
#    pyplot.set_cmap(pyplot.gray())
#    pyplot.pcolormesh(x, y, np.flipud(ArrayDicom[:,:,0,i]))
#    pyplot.show()

In [31]:
###### Get ROI file
# Get ref file
maxRow = 0
cnt = 0
for filenameROI in lstFilesROI:        
    with open(filenameROI, "rb") as f:
        roi = ijroi.read_roi(f)
    
    if maxRow < np.shape(roi)[0]:
        maxRow = np.shape(roi)[0]

# The array is sized based on 'maxRow(length of contour)'
ArrayRoi = np.zeros((numOfDataSet, max(numOfRoiData), maxRow, 2))
ArrayRoi.fill(-1)

# loop through all the ROI files
refRoi = np.zeros((numOfDataSet, max(numOfRoiData)), np.int8)
refRoi.fill(-1)

setNum = 0
roi_cnt = np.zeros(numOfDataSet)
for filenameROI in lstFilesROI:
    tmp_list = filenameROI.split('_')  
    idx = refRoiSetNum.index(tmp_list[0][-4:])
    
    tmp2_list = tmp_list[-1].split('.')
    refRoi[int(idx)][int(roi_cnt[idx])] = tmp2_list[0]
    roi_cnt[idx] = roi_cnt[idx] + 1
    
    with open(filenameROI, "rb") as f:
        roi = ijroi.read_roi(f)
    
    for i in range(np.shape(roi)[0]):
        ArrayRoi[setNum, cnt, i] = roi[i]
        
    #print ArrayRoi[setNum, cnt]
    
    cnt = cnt + 1
    if cnt == numOfRoiData[setNum]:
        setNum = setNum + 1
        cnt = 0

In [32]:
print refRoi
print roi_cnt
print numOfRoiData
print np.shape(ArrayDicom)
print np.shape(ArrayRoi)

[[11 10  9 -1 -1 -1 -1 -1 -1 -1]
 [12  9 13 11 14 15 10 -1 -1 -1]
 [12 11 13 -1 -1 -1 -1 -1 -1 -1]
 [ 8 11  9  6 12 13 10  7 -1 -1]
 [17 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [10  7 12  8  9 11 -1 -1 -1 -1]
 [13  8 10 14 15 11 16 12  9 17]
 [12 13 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 9 12 11  8 10 13 -1 -1 -1 -1]
 [13 10  7 12  6  8  9  5 11 -1]]
[  3.   7.   3.   8.   2.   6.  10.   2.   6.   9.]
(3, 7, 3, 8, 2, 6, 10, 2, 6, 9)
(384, 384, 10, 21)
(10, 10, 23, 2)


In [33]:
# Save ROI Image
for s in range(numOfDataSet):
    idx = 0
    
    while idx < numOfDataSet and refRoi[s][idx] != -1:
        img = ArrayDicom[:, :, s, refRoi[s][idx]]
        
        print ArrayRoi[s][idx]
        
        idx = idx + 1
        
    #pyplot.figure(dpi=300)    
    #pyplot.axes().set_aspect('equal', 'datalim')
    #pyplot.set_cmap(pyplot.gray())
    #pyplot.pcolormesh(x, y, np.flipud(ArrayDicom[:,:,0,i]))
    #pyplot.show()

[[ 214.  235.]
 [ 216.  242.]
 [ 211.  246.]
 [ 202.  247.]
 [ 194.  245.]
 [ 193.  245.]
 [ 202.  241.]
 [ 207.  237.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]]
[[ 220.  231.]
 [ 219.  238.]
 [ 210.  243.]
 [ 201.  243.]
 [ 196.  240.]
 [ 196.  239.]
 [ 204.  236.]
 [ 206.  230.]
 [ 210.  227.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]]
[[ 202.  155.]
 [ 193.  148.]
 [ 184.  148.]
 [ 172.  154.]
 [ 162.  164.]
 [ 168.  167.]
 [ 176.  166.]
 [ 182.  166.]
 [ 182.  159.]
 [ 181.  157.]
 [ 184.  154.]
 [ 191.  154.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1.   -1.]
 [  -1. 